In [ ]:
!pip install transformers
!pip install tensorflow-addons

In [ ]:
import h5py

h = h5py.File("/kaggle/input/mai-processed/data.hdf5", "r")
X = h["X"]
em = h["embedding_matrix"]

In [ ]:
import pickle

with open("/kaggle/input/mai-processed/y.pickle", "rb") as p:
    y = pickle.load(p)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-3B")

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

#Implement BLEU score
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=tfa.seq2seq.sequence_loss)